# Clear and transform data

In [66]:
import pandas as pd
import requests
import re

data = pd.ExcelFile('data.xlsx')
table = data.parse("Sheet1")
#table.head()

#расшиваем колонку Location на тип работы, рабочий день и географическое положение
tlocation=[]
typejob=[]
tday=[]

a = "-"
b = "-"
c = "-"
#Полный рабочий день
#Неполный рабочий день
#Можно удаленно
#

for loc in table['Location']:
    separates = loc.split(' • ')#сплитим
    if separates.count('Полный рабочий день'):
        tday.append('Полный рабочий день')
    elif separates.count('Неполный рабочий день'):
        tday.append('Неполный рабочий день')
    else:
        tday.append("-")
        
    if separates.count('Можно удаленно'):
        typejob.append('Можно удаленно')
    else:
        typejob.append("-")
    
    loc = ""
    
    for separate in separates:
        if separate != 'Полный рабочий день' and separate != 'Неполный рабочий день' and separate != 'Можно удаленно':
            if loc == "":
                loc = separate
            else:
                loc  = loc + ", " + separate
    tlocation.append (loc)
    loc = ""
    
               
    

table["Work type"] = tday
table["Remote"] = typejob
table["TLocation"] = tlocation

#table.to_excel("next.xlsx")


table = table.drop('Location', axis=1)#Удалям старое поле Locations
#table


# парсим поле Salary

smin = []
smax = []
currency = []
for salary in table['Salary']:
    salary = salary.replace(' ', '')#удаляем пробелы из строки
    currency.append (salary[-1])#сразу последний символ забиваем как валюта
    digits = re.findall(r'\d+', salary) #список с цифрами 
    
    if len(digits) == 2:
        smin.append (float(digits[0]))
        smax.append (float(digits[1]))
    elif 'от' in salary:
        smin.append (float(digits[0]))
        smax.append (0)
    elif 'до' in salary:
        smin.append (0)
        smax.append (float(digits[0]))
    else:
        smin.append (float(digits[0]))
        smax.append (0)

table['Currency'] = currency
table['Salary_min'] = smin
table['Salary_max'] = smax

#table.to_excel("next2.xlsx")


#Загружаем курс валют с сайта ЦБ РФ в формате Json()

url = 'https://www.cbr-xml-daily.ru/daily_json.js'

currency_list = requests.get(url).json()


#Определяем список уникальных валют в table, определяем наименование валют по символу, 
#а также подгружаем курс к рублю на текущую дату из currency_list

c_dict = {'₽':'RUB', '$':'USD', '€':'EUR', '₸':'KZT', '₺':'TRY', '₴':'UAH', '£':'GBP'}

curlist = table['Currency'].unique()#символы уникальных валют из таблицы
exchange_rates={} #словарь в который помещаем курс и код валюты 

for cur in curlist:
    if c_dict.get(cur) == 'RUB':
        exchange_rates[cur] = 1
    else:
        exchange_rates[cur] = float(currency_list['Valute'][c_dict.get(cur)]['Value'])/float(currency_list['Valute'][c_dict.get(cur)]['Nominal'])
#Если нет доллара в зарплатах, добавляем
if exchange_rates.get('$') == None: exchange_rates['$'] = float(currency_list['Valute']['USD']['Value'])
#exchange_rates

#table['TLocation'].get(0)
i = 0
salary_min_usd = []#минимальная ЗП в USD
salary_max_usd = []#максимальная ЗП в USD
salary_mid_usd = []#средняя ЗП в USD

#salary_min_usd.append("{:10,2f}".format(table['Salary_min'].get(i)))
for cur in table['Currency']:
    if cur == '$': #Если доллар
        if table['Salary_min'].get(i) != 0: 
            salary_min_usd.append(float(table['Salary_min'].get(i)))
        else:
            salary_min_usd.append(0)
        if table['Salary_max'].get(i) != 0: 
            salary_max_usd.append(float(table['Salary_max'].get(i)))
        else:
            salary_max_usd.append(0)
    elif cur == '₽': # если рубль
        if table['Salary_min'].get(i) != 0: 
            salary_min_usd.append(float(table['Salary_min'].get(i)/exchange_rates.get('$')))
        else:
            salary_min_usd.append(0)
        if table['Salary_max'].get(i) != 0: 
            salary_max_usd.append(float(table['Salary_max'].get(i)/exchange_rates.get('$')))
        else:
            salary_max_usd.append(0)
    else: #если другая валюта
        if table['Salary_min'].get(i) != 0:
            salary_min_usd.append(float(table['Salary_min'].get(i) * exchange_rates.get(cur)/exchange_rates.get('$')))
        else:
            salary_min_usd.append(0)
        if table['Salary_max'].get(i) != 0:
            salary_max_usd.append(float(table['Salary_max'].get(i) * exchange_rates.get(cur)/exchange_rates.get('$')))
        else:
            salary_max_usd.append(0)
    i += 1

#посчитаем среднюю ЗП в USD
for s in range(len(salary_min_usd)):
    if salary_min_usd[s] != 0 and salary_max_usd[s] != 0:
        salary_mid_usd.append((float(salary_min_usd[s]) + float(salary_max_usd[s]))/2)
    else:
        salary_mid_usd.append(float(salary_min_usd[s]) + float(salary_max_usd[s]))


table['Salary_min_usd'] = salary_min_usd
table['Salary_max_usd'] = salary_max_usd
table['Salary_mid_usd'] = salary_mid_usd

#table.to_excel('tmp.xlsx')

# узнаем уровень позиции из столбца Skills: то, что в (...)
level = []

for position in table['Skills']:
    l = re.search(r'\(\w+\)', position)
    try:#если значение не Noone
        level.append(l.group()[1:-1])
    except:#в противном случае ставим "-"
        level.append('-')
    

table.insert(2,'Level', level) #Добавляем колонку вторым номером

table.to_excel('cleardata.xlsx')#сохраняем в файл чистый